In [1]:
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from collections import namedtuple
import rasterio
%matplotlib inline
import gdal

bathymetry_path = "..." # replace input bathymetry tif path
rugosity_path = "..." # replace output rugosity tif path

In [2]:
def generate_rugosity(bathymetry_path, rugosity_path):
    with rasterio.open(bathymetry_path, 'r') as src:
        nodata = src.meta['nodata']
        bathymetry = src.read(1)

        if nodata is not None:
            bathymetry[bathymetry == nodata] = np.nan

        # Calculate gradients
        grad_x, grad_y = np.gradient(bathymetry)
        
        # Calculate the actual surface area within each cell
        actual_area = np.sqrt(grad_x**2 + grad_y**2 + 1)

        # Calculate planar area assuming each pixel is a square of 1m*1m if not specify the real dimension
        planar_area = np.ones_like(bathymetry) * 2.5 * 2.5 # (100 is the planar area)

        grad_x, grad_y, bathymetry = None, None, None # this solves out of memory issue
        
        # Calculate rugosity: actual_area / planar_area
        rugosity = actual_area / planar_area

        with rasterio.open(rugosity_path, 'w', **src.meta) as dst:
            dst.write(rugosity, 1)

generate_rugosity(bathymetry_path, rugosity_path)


In [ ]:
with rasterio.open(rugosity_path) as dataset:
    print(f"Nodata value: {dataset.meta['nodata']}")
    transform = dataset.transform
    print(f"Transform: {transform}")
    pixelSizeX = dataset.transform[0]
    pixelSizeY = -dataset.transform[4]  # Note the negative sign for ysize
    print(f"Pixel Size X: {pixelSizeX}, Pixel Size Y: {pixelSizeY}")